In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
import os

In [ ]:
# Load, chunk and index the contents of the blog.
from eurocc_v1.paths import DATA_DIR
loader = PyPDFLoader(DATA_DIR / "raw" / "PRO002-P-IT rev.01 Selezione  Autorizzazione e Qualifica dei Fornitori cg.pdf" )
docs = loader.load()

#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#splits = text_splitter.split_documents(docs)
print(DATA_DIR)
print(len(docs))
for doc in docs:
    print(doc.metadata["source"].split("/")[-1])
    #print("***************************************************************")
    #print(doc.page_content)

In [12]:
# Load, chunk and index the contents of the blog.
from eurocc_v1.paths import DATA_DIR

pdf_folder_path = DATA_DIR / "raw"
loaders = [PyPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
print(loaders)

all_documents = []
curr_doc_pages = []

for loader in loaders[:1]: # 1 loader for 1 pdf FIXME: using just one single doc for testing
    print("Loading raw document..." + loader.file_path)

    pages = loader.load()
    for page in pages:
        #print(doc.metadata["source"].split("/")[-1])
        curr_doc_pages.append(page.page_content)

    all_documents.extend(curr_doc_pages)

print([doc for doc in all_documents])

[<langchain_community.document_loaders.pdf.PyPDFLoader object at 0x7f548c0884d0>, <langchain_community.document_loaders.pdf.PyPDFLoader object at 0x7f54a47022d0>, <langchain_community.document_loaders.pdf.PyPDFLoader object at 0x7f5479b56150>]
Loading raw document.../leonardo_work/PhDLR_prod/llm_eucc/eurocc/data/raw/PRO002-P-IT rev.01 Selezione  Autorizzazione e Qualifica dei Fornitori cg.pdf
[' \n \nLe informazioni contenute nel presente documento sono di proprietà di Selex ES S.p.A.  e non possono, al pari di tale documento, essere \nriprodott e, utilizzat e o divulgat e in tutto o in parte a terzi senza preventiva autorizzazione scritta di Selex ES S.p.A.  \nIl documento è disponibile nell’Intranet Aziendale/BMS di Selex ES S.p.A. Le copie, sia in formato elettronico che cartaceo dovranno \nessere verificate, prima dell’utilizzo, con la versione vigente disponibile su Intranet.  \n© Copyright Selex ES S.p.A. 2014 - Tutti i diritti riservati  \n \nIDENTIFICATIVO : PRO0 02-P-IT Rev. 0

In [12]:
from sentence_transformers import SentenceTransformer
from llama_index.core.node_parser import SentenceSplitter
#from bson.objectid import ObjectId
import faiss
import numpy as np
import os

#def create_vector_store(db, index_type, chunk_size, chunk_overlap,  embedding_model_name):
def create_vector_store(index_type, chunk_size, chunk_overlap,  embedding_model_name):

    embedding_model = SentenceTransformer(embedding_model_name, trust_remote_code=True)

    embedding_dim = embedding_model.encode(["This is a test"]).shape[1]
    print("Embedding dim: ", embedding_dim)
    
    #coll_demetra = db["demetraLeggi"]

    text_splitter = SentenceSplitter(chunk_size=chunk_size,
                                     chunk_overlap=chunk_overlap)

    list_texts = []
    _id_faiss = 0

    #for ind, doc in enumerate(coll_demetra.find()[:]):
    for ind, doc in enumerate(docs):
        
        #_id = ObjectId(doc["_id"])
        _id = ind # TODO: to replace with a unique identifier for the document/procedure
        
        # titolo_clean = clean_title(doc['titolo'])
        # print(titolo_clean)
        titolo_clean = doc.metadata["source"].split("/")[-1]
        list_texts.append({"_id_proc": _id, "_id_faiss": _id_faiss, "testo": titolo_clean})
        _id_faiss += 1
        
        #list_texts.append({"_id_legge": _id, "_id_faiss": _id_faiss, "testo": doc['legge']})
        #_id_faiss += 1

        text = doc.page_content
            
        # if text == "":
        #    if 'testo_orig' in doc.keys() and 'testo' in doc['testo_orig'].keys():
        #       print(ind)
        #        text = doc['testo_orig']['testo']

        text_splitted = text_splitter.split_texts([text])[:2] # FIXME: taking just 2 chunks to test workflow
        
        for elem in text_splitted:
            list_texts.append({"_id_proc": _id, "_id_faiss": _id_faiss, "testo": elem}) #elem.replace("\n", " ")})
    #        list_ids.append({"_id_mongo": _id, "_id_faiss": _id_faiss})
            _id_faiss += 1

    #delete previous collection and insert into mongo
#    coll_dementra_leggi_chunks_name = "demetraLeggiChunks_" + str(chunk_size) + "_" +  str(chunk_overlap) + "_" + index_type + "_" + embedding_model_name.replace("/", "_")
    index_name = "index_proc_" + str(chunk_size) + "_" + str(chunk_overlap) + "_" + index_type + "_" + embedding_model_name.replace("/", "_")
    #coll_dementra_leggi_chunks = db[index_name]
    #coll_dementra_leggi_chunks.drop()

    #res = coll_dementra_leggi_chunks.insert_many(list_texts)
    #print("inserted:", len(res.inserted_ids), "items")

    #create index on _id_faiss for performance and uniqueness
    #coll_dementra_leggi_chunks.create_index([("_id_faiss", pymongo.ASCENDING)], unique=True)
    #print("created index for:", index_name)
    
    if index_type == "L2":
        print("L2 index")
        index = faiss.IndexIDMap2(faiss.IndexFlatL2(embedding_dim))
    elif index_type == "IP":
        print("IP index")
        index = faiss.IndexIDMap2(faiss.IndexFlatIP(embedding_dim))
    else:
        index = None
        
    for ind, item in enumerate(list_texts[0:]):
        if ind % 1000 == 0:
            print(ind)
        # print(item['_id'])
        text = item['testo']
        #embedding = embedding_model.encode([text]) # FIXME: using fake embedding to test failure point
        embedding = np.array([[0] * 1024])
        if index_type == "IP":
            faiss.normalize_L2(embedding)
        
        index.add_with_ids(embedding, np.array([ind]))

#    index_name = "index_demetra_" + str(chunk_size) + "_" + str(chunk_overlap) + "_" + index_type + ".faiss"
#    index_name = "index_demetra_" + str(chunk_size) + "_" + str(chunk_overlap) + "_" + index_type + "_" + embedding_model_name.replace("/", "_") + ".faiss"

    faiss.write_index(index, os.path.join("indices", index_name + ".faiss"))

    print("num tot chunks:", index.ntotal)

In [14]:
#embedding_models = ["BAAI/bge-m3", "intfloat/multilingual-e5-large-instruct", "nomic-ai/nomic-embed-text-v1", "nickprock/sentence-bert-base-italian-uncased"]
#chunk_sizes = [64, 128, 256, 512, 1024]
#chunk_overlaps = [20, 50]
#index_types = ["L2", "IP"]

embedding_models = ["BAAI/bge-m3"]
chunk_sizes = [64]
chunk_overlaps = [20]
index_types = ["L2"]


for embedding_model in embedding_models:
    for index_type in index_types:
        for chunk_size in chunk_sizes:
            for chunk_overlap in chunk_overlaps:
                print("creating index:", embedding_model, index_type, chunk_size, chunk_overlap)
                #create_vector_store(db, index_type, chunk_size, chunk_overlap,  embedding_model)
                create_vector_store(index_type, chunk_size, chunk_overlap,  embedding_model)
                print()

creating index: BAAI/bge-m3 L2 64 20
Embedding dim:  1024
L2 index
0
num tot chunks: 126



In [18]:
from langchain_community.embeddings import OpenAIEmbeddings
# FIXME: need to change OpenAIEmbeddings with ours
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=""))

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

prompt = """You are an expert rgarding the company internal procedure documents. You need to answer the question related to the internal procedure documents.
Given below is the context and question of the user.
context = {context}
question = {question}
"""

prompt = ChatPromptTemplate.from_template(prompt)

#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key="")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
#    | llm
#    | StrOutputParser()
)

AttributeError: 'list' object has no attribute 'embed_documents'

In [ ]:
rag_chain.invoke("What is the goal of the procedure PRO002 ?")

In [ ]:
query = "Tell me more about law number 30"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

In [ ]:
docs_and_scores = vectorstore.similarity_search_with_score(query)
docs_and_scores[0]